# Count CSV Rows in Labels Subfolders

This notebook counts rows (excluding headers) in CSV files within specified labels subfolders (e.g., `labels/jaw` or `labels/tip`).

In [11]:
from utils import scan_csv_in_labels_subfolders, display_csv_scan_results
import pandas as pd

# Configure pandas to display all rows
pd.set_option('display.max_rows', None)

In [12]:
# Root directory containing folders with CSV files
ROOT_DIR = r'C:\Users\wanglab\Desktop\Tip+Curve+Ryan'

# Specify which subfolder within 'labels' to search for CSV files
# Examples: 'jaw', 'tip', 'tongue', etc.
SUBFOLDER_NAME = 'tip'

In [13]:
# Scan the directory structure
scan_data = scan_csv_in_labels_subfolders(ROOT_DIR, SUBFOLDER_NAME)

# Display formatted results
df_with_totals, df_valid = display_csv_scan_results(scan_data)

# Show the full table
display(df_with_totals)

Scanning 65 folders in C:\Users\wanglab\Desktop\Tip+Curve+Ryan
Looking for CSV files in: labels/tip/

Found CSV files in 65 cases
Missing or failed: 0
Total rows (excluding headers): 4284

CSV Row Counts by Folder:

Statistics for valid CSV files:
  Min rows: 0
  Max rows: 493
  Mean rows: 65.91
  Median rows: 35.00
Found CSV files in 65 cases
Missing or failed: 0
Total rows (excluding headers): 4284

CSV Row Counts by Folder:

Statistics for valid CSV files:
  Min rows: 0
  Max rows: 493
  Mean rows: 65.91
  Median rows: 35.00


,Folder,CSV File,Rows (excluding header),Status
0,10_bottom,tip_bottom_10.csv,27,Success
1,10_side,tip_side_10.csv,34,Success
2,11_bottom,tip_bottom_11.csv,20,Success
3,11_side,tip_side_11.csv,39,Success
4,12_bottom,tip_bottom_12.csv,20,Success
5,12_side,tip_side_12.csv,55,Success
6,13_bottom,tip_bottom_13.csv,14,Success
7,13_side,tip_side_13.csv,61,Success
8,14_bottom,tip_bottom_14.csv,13,Success
9,14_side,tip_side_14.csv,29,Success
